# 다중 회귀

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures # 변환기 (데이터 바꾸는 용도) 수의 조합 생성
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression # 선형 회귀
from sklearn.linear_model import Ridge # 릿지 회귀
from sklearn.linear_model import Lasso # 라쏘 회귀

df = pd.read_csv('https://bit.ly/perch_csv')
perch_full = df.to_numpy()

perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)

poly = PolynomialFeatures()
poly.fit([[2, 3]])
print(poly.transform([[2, 3]])) # fit & transform 항상 같이 나온다

poly = PolynomialFeatures(include_bias=False) # bias 1 값을 제외할 수도 있다.
poly.fit([[2, 3]])
print(poly.transform([[2, 3]]))

# 변환 시작
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
print(train_poly.shape) # (42, 9) : 42개 데이터의 9개 특성치
poly.get_feature_names()

test_poly = poly.transform(test_input) # test set은 학습 없이 변환한다.

# 다중 회귀 모델 훈련 시작

lr = LinearRegression()
lr.fit(train_poly, train_target)
print(lr.score(train_poly, train_target)) # 0.9903183436982124
print(lr.score(test_poly, test_target))   # 0.9714559911594134
# 다항 회귀까지 했을 때보다 좋은 학습 결과가 나왔다

# 그렇다면, 특성치가 많으면 무조건 좋을까?
poly = PolynomialFeatures(degree=5, include_bias=False) # 5차항으로 생성

poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
print(train_poly.shape) # (42, 55) : 42개 데이터의 55개 특성치...잘못된 데이터

lr.fit(train_poly, train_target)
print(lr.score(train_poly, train_target)) # 0.9999999999991097
print(lr.score(test_poly, test_target))   # -144.40579242684848
# 과적합으로 인해 실패한 케이스


# 규제

회귀에 규제를 하기 위해서는 정규화를 해야한다. 복잡도에 적절한 규제 강도를 탐색한다.

In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import PolynomialFeatures # 변환기 (데이터 바꾸는 용도) 수의 조합 생성
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression # 선형 회귀
# from sklearn.linear_model import Ridge # 릿지 회귀
# from sklearn.linear_model import Lasso # 라쏘 회귀
import matplotlib.pyplot as plt

# 표준화
ss = StandardScaler() # 특성을 표준 점수로 자동 변환하는 변환기의 일종(평균과 표준편차를 이용한 표준 점수 산출)
ss.fit(train_poly) # 훈련 세트에 적용한 객체를 테스트 세트에 적용

train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly) # 표준 점수로 훈련/테스트 세트 변환

print(train_scaled)

# 규제1. 릿지 회귀
# 가중치의 제곱을 벌칙으로 사용, L2 규제
# 규제 강도 조절을 위해 하이퍼 파라미터를 조정한다.

ridge = Ridge()
ridge.fit(train_scaled, train_target) # target은 스케일되지 않은 그대로 사용
print(ridge.score(train_scaled, train_target)) # 0.9896101671037343
print(ridge.score(test_scaled, test_target))   # 0.9790693977615391
# 규제를 통한 과대 적합 해소

# 적절한 규제 강도 찾기 : alpha 값의 변화에 따른 결정계수(R^2) 탐색
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
train_score = []
test_score = []

for alpha in alpha_list:
  ridge = Ridge(alpha = alpha)
  ridge.fit(train_scaled, train_target) # 릿지 모델 훈련
  train_score.append(ridge.score(train_scaled, train_target))
  test_score.append(ridge.score(test_scaled, test_target))

plt.plot(np.log10(alpha_list), train_score)
plt.plot(np.log10(alpha_list), test_score)
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()

In [ ]:
# 규제2. 라쏘 회귀
# 가중치의 절대값을 벌칙으로 사용, L1 규제

lasso = Lasso()
lasso.fit(train_scaled, train_target)

print(lasso.score(train_scaled, train_target)) # 0.989789897208096
print(lasso.score(test_scaled, test_target))  # 0.9800593698421883

alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]
train_score = []
test_score = []

for alpha in alpha_list:
  lasso = Lasso(alpha=alpha, max_iter=10000)
  lasso.fit(train_scaled, train_target)
  train_score.append(lasso.score(train_scaled, train_target))
  test_score.append(lasso.score(test_scaled, test_target))

plt.plot(np.log10(alpha_list), train_score)
plt.plot(np.log10(alpha_list), test_score)
plt.show()

# 라쏘 회귀는 특성의 가중치를 0으로 만들 수도 있다. (특성 무시) 
lasso = Lasso(alpha=10)
lasso.fit(train_scaled, train_target)

print(lasso.score(train_scaled, train_target)) # 0.9888067471131867
print(lasso.score(test_scaled, test_target))   # 0.9824470598706695

print(np.sum(lasso.coef_ == 0)) # 특성 가중치가 0인 것이 40개 존재 => 유용한 특성을 골라내는 용도로 사용

# 로지스틱 회귀

분류하는데 사용하는 알고리즘. (이름에서 혼동하지 말 것)

In [ ]:
fish = pd.read_csv('https://bit.ly/fish_csv')
fish.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

# K-최근접 이웃 분류의 확률 예측

kn = KNeighborsClassifier(n_neighbors = 3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target)) # 0.8907563025210085
print(kn.score(test_scaled, test_target))   # 0.85

print(kn.predict(test_scaled[:5])) # 예측 시작
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4)) # predict_proba() : 각 클래스별 확률값 반환

distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

In [ ]:
# 로지스틱 회귀

# sigmoid 함수 식
z = np.arange(-5, 5, 0.1)
phi = 1 / (1 + np.exp(-z))

plt.plot(z, phi)
plt.show()

# 로지스틱 회귀 (이진 분류)

In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.special import expit

bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt') # boolean index retrun
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

print(lr.predict(train_bream_smelt[:5]))
print(lr.predict_proba(train_bream_smelt[:5]))

# 로지스틱 회귀 식(계수, 절편) 확인
print(lr.coef_, lr.intercept_) # 특성 5개에 대한 가중치(계수) 5개 + 절편 1개

decisions = lr.decision_function(train_bream_smelt[:5]) # 입력에 대한 표준 점수 z 출력 함수(평균에서 몇 표준편차만큼?)
print(expit(decisions)) # 표준점수 z를 sigmoid함수 expit에 입력하여 확률 산출 (양성일 확률)
# [0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]

# 로지스틱 회귀 (다중 분류)

In [ ]:
lr = LogisticRegression(C=20, max_iter=1000) # C(alpha 대신 사용)가 클수록 규제 약함, 반복횟수 100(기본) -> 1000
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target)) # 0.9327731092436975
print(lr.score(test_scaled, test_target))   # 0.925

proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))
print(lr.coef_.shape, lr.intercept_.shape) # (7,5) (7,) : 7개 항에 5개씩 특성치가 존재하고, 각 항별 식의 절편 개수


# 소프트 맥스 함수 : 7개의 클래스(다중 분류)를 각각 0~1 사이 확류로 표시
# sigmoid -> 이진 분류
# softmax -> 다중 분류

decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

from scipy.special import softmax

proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3)) # 위의 결과와 같다

# SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier

fish = pd.read_csv('https://bit.ly/fish_csv')

fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

sc = SGDClassifier(loss='log', max_iter=10, random_state=42) # 확률적 경사하강법
sc.fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target)) # 0.773109243697479
print(sc.score(test_scaled, test_target))   # 0.775

sc.partial_fit(train_scaled, train_target) # 이전 학습 결과를 유지하면서 1 에포크 추가 훈련

print(sc.score(train_scaled, train_target)) # 0.8151260504201681
print(sc.score(test_scaled, test_target))   # 0.85
# 향상되었다 -> 얼마나 partial_fit()을 반복할까? -> 에포크 횟수 조절

# 에포크와 과대/과소적합

In [ ]:
sc = SGDClassifier(loss='log', random_state=42)

train_score = []
test_score = []

classes = np.unique(train_target)

for _ in range(0, 300):
  sc.partial_fit(train_scaled, train_target, classes=classes)

  train_score.append(sc.score(train_scaled, train_target))
  test_score.append(sc.score(test_scaled, test_target))

plt.plot(train_score)
plt.plot(test_score)
plt.show()

# 손실 함수 log 사용
sc = SGDClassifier(loss='log', max_iter=100, tol=None, random_state=42)
sc.fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target)) # 0.957983193277311
print(sc.score(test_scaled, test_target))   # 0.925

# 손실 함수 hinge 사용
sc = SGDClassifier(loss='hinge', max_iter=100, tol=None, random_state=42)
sc.fit(train_scaled, train_target)

print(sc.score(train_scaled, train_target)) # 0.9495798319327731
print(sc.score(test_scaled, test_target))   # 0.925

# 결정 트리

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

# 전처리
wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

lr = LogisticRegression()
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))


train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

# 결정 트리
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_scaled, train_target)

print(dt.score(train_scaled, train_target)) # 0.996921300750433
print(dt.score(test_scaled, test_target))   # 0.8592307692307692

plt.figure(figsize=(10,7))
plot_tree(dt)
plt.show()

plt.figure(figsize=(10, 7))
plot_tree(dt, max_depth=1, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()


# 가지치기

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_scaled, train_target)

print(dt.score(train_scaled, train_target)) # 0.8454877814123533
print(dt.score(test_scaled, test_target))   # 0.8415384615384616

plt.figure(figsize=(20,15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

In [ ]:
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(train_input, train_target)

print(dt.score(train_input, train_target)) # 0.8454877814123533
print(dt.score(test_input, test_target))   # 0.8415384615384616

plt.figure(figsize=(20,15))
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()

print(dt.feature_importances_) # [0.12345626 0.86862934 0.0079144 ]

In [ ]:
# 확인 문제

dt = DecisionTreeClassifier(min_impurity_decrease=0.0005, random_state=42)
dt.fit(train_input, train_target)

print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

plt.figure(figsize=(20,15), dpi=300)
plot_tree(dt, filled=True, feature_names=['alcohol', 'sugar', 'pH'])
plt.show()